https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation,\
Configuration, Action, row_col, adjacent_positions, translate, min_distance,random_agent, GreedyAgent

from kaggle_environments import make
import numpy as np
from random import choice

import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
from collections import deque
import time

In [ ]:
# ACTIONS = [e.name for e in Action]
ACTIONS = ['NORTH', 'SOUTH', 'WEST', 'EAST']

In [ ]:
env = make("hungry_geese")
display(env.reset())

trainer = env.train([None, "greedy", "greedy", "greedy"])
trainer.reset()

In [ ]:

def DQNet():
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(7,11,17)))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,5), activation='relu'))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
#     model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='linear'))
    
    model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    
    return model

model = DQNet()
# model.build(input_shape=(None, 7, 11, 17))
model.summary()

In [ ]:
def centerize(b):
    dy, dx = np.where(b[0])
    if len(dy) == 0 and len(dx) == 0:
        return b
    centerize_y = (np.arange(0,7)-3+dy[0])%7
    centerize_x = (np.arange(0,11)-5+dx[0])%11
    
    b = b[:, centerize_y,:]
    b = b[:, :,centerize_x]
    
    return b

def make_input(obs, prev_obs):
#     print(obs)
    b = np.zeros((17, 7 * 11), dtype=np.float32)
#     obs = obses[-1]

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs['index']) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs['index']) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs['index']) % 4, pos] = 1
            
    # previous head position
    if prev_obs is not None:
        obs_prev = prev_obs
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[12 + (p - obs['index']) % 4, pos] = 1

    # food
    for pos in obs['food']:
        b[16, pos] = 1
        
    b = b.reshape(-1, 7, 11)
    b = centerize(b)
    b = np.transpose(b, (1,2,0))

    return b

In [ ]:
def train(env, replay_memory, model, target_model, done):
    
    learning_rate = 0.7 # Learning rate
    discount_factor = 0.618
    batch_size = 512 * 2

    MIN_REPLAY_SIZE = 3000
    if len(replay_memory) < MIN_REPLAY_SIZE:
        return

    
    mini_batch = random.sample(replay_memory, batch_size)
    
    rewards = np.array([transition[2] for transition in mini_batch])
    
    current_states = np.array([transition[0] for transition in mini_batch])
    current_qs_list = model.predict(current_states)
    
    
    new_current_states = np.array([transition[3] for transition in mini_batch])
    
#     non_final_states = np.array([transition[3] for transition in mini_batch if not transition[4]])
#     non_final_index_mask = np.array([False if transition[4] else True  for transition in mini_batch])
    
    
#     future_qs_list = tf.zeros(batch_size)
    
    future_qs_list = target_model.predict(new_current_states)
    
#     next_state_reward = future_qs_list*discount_factor + rewards

    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done) in enumerate(mini_batch):
        if not done:
            max_future_q = reward + discount_factor * np.max(future_qs_list[index])
        else:
            max_future_q = reward

#         current_qs = current_qs_list[index]
        current_qs = [0]*4
        current_qs[action] = (1 - learning_rate) * current_qs[action] + learning_rate * max_future_q

        X.append(observation)
        Y.append(current_qs)
#     print(Y)
    model.fit(np.array(X), np.array(Y), batch_size=batch_size, verbose=0, shuffle=True)

In [ ]:
def main():
    trainer = env.train([None, "greedy", "greedy", "greedy"])
    trainer.reset()
    is_render = False
    
    epsilon = 1 # Epsilon-greedy algorithm in initialized at 1 meaning every step is random at the start
    max_epsilon = 1 # You can't explore more than 100% of the time
    min_epsilon = 0.01 # At a minimum, we'll always explore 1% of the time
    decay = 0.01
    
    # An episode a full game
    train_episodes = 15000
    test_episodes = 100

    # 1. Initialize the Target and Main models
    # Main Model (updated every 4 steps)
    model = DQNet()
    # Target Model (updated every 100 steps)
    target_model = DQNet()
    target_model.set_weights(model.get_weights())

    replay_memory = deque(maxlen=20000)

    target_update_counter = 0

    # X = states, y = actions
    X = []
    y = []
    
#     observation_list = []
    steps_to_update_target_model = 0
    prev_obs = None

    for episode in range(train_episodes):
        total_training_rewards = 0
        observation = trainer.reset()
#         observation_list = []
        prev_obs = None
        done = False
#         print("***********New Eposide **************")
        while not done:
#             observation_list.append(observation)
#             prev_obs = observation
            encoded_observation = make_input(observation, prev_obs)
            
            steps_to_update_target_model += 1
            if is_render:
                env.render(mode="ipython", width=500, height=450)

            random_number = np.random.rand()
            # 2. Explore using the Epsilon Greedy Exploration Strategy
            if random_number <= epsilon or len(replay_memory) < 5000:
                # Explore
                g_agent = GreedyAgent(Configuration({'rows': 7, 'columns': 11}))
                action = g_agent(Observation(observation))
                
                action = ACTIONS.index(action)
            else:
                # Exploit best known action
                # model dims are (batch, env.observation_space.n)
                encoded_reshaped = encoded_observation.reshape(-1,7,11,17)
                
                predicted = model.predict(encoded_reshaped)
                action = np.argmax(predicted)
#             print(action)
            new_observation, reward, done, info = trainer.step(ACTIONS[action])
            if done and reward == 0:
#                 print("*****************Done*******************")
                reward = -1000
                
            prev_obs = observation
            new_encoded_observation = make_input(new_observation, prev_obs)
            replay_memory.append([encoded_observation, action, reward, new_encoded_observation, done])

            # 3. Update the Main Network using the Bellman Equation
            if steps_to_update_target_model % 4 == 0 or done:
#                 print("***********Train************")
                train(env, replay_memory, model, target_model, done)
            
            observation = new_observation
            total_training_rewards += reward

            if done:
#                 print('Total training rewards: {} after n steps = {} with final reward = {}'.format(total_training_rewards, episode, reward))
                total_training_rewards += 1

                if steps_to_update_target_model >= 100:
                    print('Copying main network weights to the target network weights')
                    target_model.set_weights(model.get_weights())
                    steps_to_update_target_model = 0
                break

        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)
        
        if episode % 300 == 0:
            model.save('my_model.h5')
#     env.close()
    model.save('my_model.h5')

if __name__ == '__main__':
    main()

In [ ]:
# obs_list = []
# obs = trainer.reset()
# obs_list.append(obs)
# model = tf.keras.models.load_model('my_model.h5')

# done = False
# total_reward = 0
# while not done:
#     obs_list.append(obs)
#     encoded_observation = make_input(obs_list)
#     encoded_reshaped = encoded_observation.reshape(-1,7,11,17)
                
#     predicted = model.predict(encoded_reshaped)
# #     print(predicted)
#     action = np.argmax(predicted)
# #     print(action)
#     new_obs, reward, done, info = trainer.step(ACTIONS[action])
# #     env.render(mode="ipython", width=500, height=450)
    
#     obs = new_obs
#     total_reward += reward
    
#     if done:
#         print(f'Total reward: {total_reward}')
        
   


In [ ]:
%%writefile main.py

import sys
from pathlib import Path
import numpy as np
import tensorflow as tf

p = Path('/kaggle_simulations/agent/')
if p.exists():
    sys.path.append(str(p))
else:
    p = Path('__file__').resolve().parent

In [ ]:
%%writefile -a main.py

import tensorflow as tf
import numpy as np
import random

def centerize(b):
    dy, dx = np.where(b[0])
    if len(dy) == 0 and len(dx) == 0:
        return b
    
    centerize_y = (np.arange(0,7)-3+dy[0])%7
    centerize_x = (np.arange(0,11)-5+dx[0])%11
    
    b = b[:, centerize_y,:]
    b = b[:, :,centerize_x]
    
    return b

def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs['index']) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs['index']) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs['index']) % 4, pos] = 1
            
    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[12 + (p - obs['index']) % 4, pos] = 1

    # food
    for pos in obs['food']:
        b[16, pos] = 1
        
    b = b.reshape(-1, 7, 11)
    b = centerize(b)
    b = np.transpose(b, (1,2,0))

    return b

model = tf.keras.models.load_model(str(p/'my_model.h5'))

obses = []

def agent(obs_dict, config_dict):
    obses.append(obs_dict)

    X_test = make_input(obses)
#     X_test = np.transpose(X_test, (1,2,0))
    X_test = X_test.reshape(-1,7,11,17) # channel last.
    
    # avoid suicide
#     obstacles = X_test[:,:,:,[8,9,10,11,12]].max(axis=3) - X_test[:,:,:,[4,5,6,7]].max(axis=3) # body + opposite_side - my tail
#     obstacles = np.array([obstacles[0,2,5], obstacles[0,4,5], obstacles[0,3,4], obstacles[0,3,6]])
    
    y_pred = model.predict(X_test) 

    
    actions = ['NORTH', 'SOUTH', 'WEST', 'EAST']
    return actions[np.argmax(y_pred)]

In [ ]:
from kaggle_environments import make
env = make("hungry_geese", debug=True)

env.reset()
env.run(['main.py', 'main.py','main.py','main.py'])
env.render(mode="ipython", width=500, height=450)

In [ ]:
!tar -czf submission.tar.gz main.py my_model.h5

In [ ]:
# str(p/'model')